# Setup

In [4]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import yaml
import os

with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

os.environ['OPENAI_API_KEY'] = config['OPENAI_API_KEY']

openai = OpenAI(model_name='gpt-3.5-turbo-instruct', temperature=0)

# RouterChain

In [6]:
chain = (
    PromptTemplate.from_template(
        """
        Classifique a pergunta do usuário em uma das seguintes categorias:
        - Assuntos Financeiros
        - Suporte Técnico
        - Atualização de Cadastro
        - Outras Informações

        Pergunta: {query}
        Classificação:
        """
    )
    | openai
    | StrOutputParser()
)

print(chain)

first=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='\n        Classifique a pergunta do usuário em uma das seguintes categorias:\n        - Assuntos Financeiros\n        - Suporte Técnico\n        - Atualização de Cadastro\n        - Outras Informações\n\n        Pergunta: {query}\n        Classificação:\n        ') middle=[OpenAI(client=<openai.resources.completions.Completions object at 0x73aa71685810>, async_client=<openai.resources.completions.AsyncCompletions object at 0x73aa719f7310>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))] last=StrOutputParser()


In [8]:
financial_chain = PromptTemplate.from_template(
    """
    Você é um especialista financeiro.
    Sempre responda às perguntas começando com "Bem-vindo ao Suporte Financeiro".
    Responda à pergunta do usuário:
    Pergunta: {query}
    Resposta:
    """
) | openai
tech_support_chain = PromptTemplate.from_template(
    """
    Você é um especialista em suporte técnico.
    Sempre responda às perguntas começando com "Bem-vindo ao Suporte Técnico".
    Ajude o usuário com seu problema técnico.
    Pergunta: {query}
    Resposta:
    """
) | openai
update_registration_chain = PromptTemplate.from_template(
    """
    Você é um representante de atendimento ao cliente.
    Sempre responda às perguntas começando com "Bem-vindo ao Suporte de Cadastro".
    Guie o usuário na atualização de suas informações de cadastro.
    Pergunta: {query}
    Resposta:
    """
) | openai
other_info_chain = PromptTemplate.from_template(
    """
    Você é um assistente de informações gerais.
    Sempre responda às perguntas começando com "Bem-vindo ao Suporte Geral".
    Forneça informações ao usuário sobre sua pergunta.
    Pergunta: {query}
    Resposta:
    """
) | openai

In [11]:
user_input = {'query':'Como faço para redefinir minha senha?'}
classification = chain.invoke(user_input)
print(classification)

 Suporte Técnico


In [12]:
def route(info):
    topic = info['topic'].lower()
    if 'financeiro' in topic:
        return financial_chain
    elif 'técnico' in topic:
        return tech_support_chain
    elif 'atualização' in topic or 'cadastro' in topic:
        return update_registration_chain
    else:
        return other_info_chain

In [13]:
response_chain = route({'topic':classification})
response = response_chain.invoke(user_input)

In [14]:
print(response)

Bem-vindo ao Suporte Técnico. Para redefinir sua senha, você precisará acessar a página de login e clicar na opção "Esqueci minha senha". Em seguida, siga as instruções na tela para redefinir sua senha. Se você tiver problemas durante o processo, não hesite em entrar em contato conosco novamente para obter ajuda adicional.


In [15]:
def test_router(question):
    user_input = {'query':question}
    classification = chain.invoke(user_input)
    response_chain = route({'topic':classification})
    response = response_chain.invoke(user_input)
    print(response)

In [16]:
test_router('Como posso pagar uma fatura atrasada?')

Bem-vindo ao Suporte Financeiro. Como posso ajudá-lo?
    Para pagar uma fatura atrasada, você pode acessar o seu perfil no nosso site e selecionar a opção de pagamento. Lá, você poderá escolher a forma de pagamento e inserir os dados necessários para efetuar o pagamento. Se preferir, também é possível entrar em contato com a nossa central de atendimento e realizar o pagamento por telefone.


In [17]:
test_router('Preciso alterar meu endereço de e-mail?')

Bem-vindo ao Suporte de Cadastro. Para alterar seu endereço de e-mail, siga os seguintes passos:
    1. Acesse o site da nossa empresa e faça login na sua conta.
    2. Clique na opção "Minha Conta" ou "Configurações".
    3. Procure pela opção "Informações de Contato" ou "Dados Pessoais".
    4. Clique em "Editar" ou no ícone de lápis ao lado do seu endereço de e-mail atual.
    5. Insira o novo endereço de e-mail desejado e clique em "Salvar".
    Pronto! Seu endereço de e-mail foi atualizado com sucesso. Caso tenha algum problema, não hesite em entrar em contato conosco novamente.


In [18]:
test_router('Qual é a missão da empresa?')

Bem-vindo ao Suporte Geral. A missão da empresa é fornecer produtos e serviços de alta qualidade para atender às necessidades de nossos clientes e contribuir para o crescimento e sucesso de suas empresas. Nós nos esforçamos para ser uma empresa líder em inovação, sustentabilidade e responsabilidade social.
